## Ambiente Inicial

In [1]:
# librerias
import os
import pandas as pd
import numpy as np


In [2]:
script = os.getcwd()
file = os.path.join(script, "data", "BD_IncendiosSNIF_2015-2023.csv")
df = pd.read_csv(file, encoding="latin-1")

C:\Users\Franc\AppData\Local\Temp\ipykernel_4236\4104928706.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding="latin-1")


In [26]:
df.sample(3)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
40764,2021,19.142886,-99.013539,Ciudad de México,Milpa Alta,Centro,B.C. San Lorenzo Tlacoyucan,02/01/2021,02/01/2021,1 Día,...,0.0,0.5,2.5,0.0,3.0,0 a 5 Hectáreas,12:05:00 a. m.,01:25:00 a. m.,03:30:00 a. m.,Actividades Ilícitas
7738,2016,19.142778,-101.618417,Michoacán,Turicato,Occidente,Ej. Turicato,12/04/2016,12/04/2016,1 Día,...,1.1,3.0,4.0,0.0,10.1,6 a 10 Hectáreas,12:10:00 a. m.,01:00:00 a. m.,07:00:00 a. m.,Actividades Ilícitas
41241,2021,19.348333,-99.429192,Ciudad de México,Lerma,Centro,Ejido Santiago Analco / El Encinal,06/02/2021,06/02/2021,1 Día,...,0.0,2.5,1.5,0.0,4.0,0 a 5 Hectáreas,12:05:00 a. m.,01:00:00 a. m.,06:30:00 a. m.,Actividades Ilícitas


## Exploración de datos y limpieza

Comenzaremos por eliminar aquellos espacios en blanco y cambiarlos por "0" para tratarlos después.

In [3]:
df.replace(" ", 0, inplace=True)
df["Renuevo"] = df["Renuevo"].astype(float) 

Ahora corrobaremos que nuestros datos hayan sido leídos correctamente y tengamos los tipos de datos correctos

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62441 entries, 0 to 62440
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               62441 non-null  int64  
 1   Latitud           62441 non-null  float64
 2   Longitud          62441 non-null  float64
 3   Estado            62441 non-null  object 
 4   Municipio         62441 non-null  object 
 5   Región            62441 non-null  object 
 6   Predio            62441 non-null  object 
 7   Fecha Inicio      62438 non-null  object 
 8   Fecha Termino     62441 non-null  object 
 9   Duración días     62440 non-null  object 
 10  Tipo de incendio  62441 non-null  object 
 11  Tipo Vegetación   62441 non-null  object 
 12  Régimen de fuego  62441 non-null  object 
 13  Tipo impacto      62441 non-null  object 
 14  Arbolado Adulto   54758 non-null  float64
 15  Renuevo           61506 non-null  float64
 16  Arbustivo         61918 non-null  float6

En esta sección trataremos los valores categoricos para normalizar nuestras respuestas, ya que hay mucha variabilidad entre respuestas.

In [6]:
estados = {
    'Ciudad De México':'Ciudad de México',
    'México':'Ciudad de México'
}

df["Estado"] = df["Estado"].replace(estados) 

In [7]:
municipios = { 
    "Alamos":"Álamos",
    "Angel Albino Corzo":"Ángel Albino Corzo"
}
df['Municipio'] = df['Municipio'].replace(municipios)

In [8]:
df["Duración días"] = df["Duración días"].replace("#¡REF!", '0') 

In [9]:
incendios = {
    'superficial':'Superficial',
    'SUPERFICIAL':'Superficial',
    'mixto':'Mixto',
    '0':"Desconocido"
}
df["Tipo de incendio"] = df["Tipo de incendio"].replace(incendios)

In [10]:
vegetacion = {
    '0':"Desconocida",
    "Bosque Mesofilo - BM": "Bosque Mesófilo - BM",
    "Matorral Desertico Micrófilo":"Matorral Desértico Micrófilo",
    "Matorral Desertico Rosetófilo":"Matorral Desértico Rosetófilo",
    "Matorral Sarco-Crasicaule":"Matorral Sarcocrasicaule",
    "Pastizal Gypsófilo":"Pastizal Gipsófilo",
    "Vegetación Halofila Hidrófila":"Vegetación Halófila Hidrófila",
}
df['Tipo Vegetación'] = df['Tipo Vegetación'].replace(vegetacion)

In [11]:
impactos = {'Impacto Mínimo':"Impacto Minimo",
'impacto mínimo':"Impacto Minimo",
'impacto Mínimo':"Impacto Minimo",
'Impacto mínimo':"Impacto Minimo",
'Impacto MÍnImo':"Impacto Minimo",
'impacto moderado':'Impacto Moderado',
'Impacto moderado':'Impacto Moderado',
'impacto severo':'Impacto Severo',
'Impacto severo':'Impacto Severo',
'0':"Impacto Nulo",
}
df["Tipo impacto"] = df["Tipo impacto"].replace(impactos)

In [12]:
df["Tamaño"] = df["Tamaño"].replace({'0':"Desconocido"})

In [13]:
causas = {
    'fumadores':"Fumadores",
    'actividades pecuarias':"Actividades Pecuarias",
    'Actividades pecuarias':"Actividades Pecuarias",
    'cazadores':"Cazadores",
    'actividades ilícitas':"Actividades Ilícitas",
    'Quema de basureros':"Quema de Basureros",
    'Otras actividades productivas':"Otras Actividades Productivas",
    'Residuos de aprovechamiento forestal':"Residuos de Aprovechamiento Forestal",
    'fogatas':"Fogatas",
    'Actividades agrícolas':"Actividades Agrícolas",
    'DESCONOCIDAS':"Desconocidas",
    'Actividades ilícitas': "Actividades Ilícitas"
}
df["Causa"] = df["Causa"].replace(causas) 

Una vez que ya estén normalizadas las respuestas procederemos a limpiar los datos nulos o con valores "0" que asginamos anteriormente.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62441 entries, 0 to 62440
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               62441 non-null  int64  
 1   Latitud           62441 non-null  float64
 2   Longitud          62441 non-null  float64
 3   Estado            62441 non-null  object 
 4   Municipio         62441 non-null  object 
 5   Región            62441 non-null  object 
 6   Predio            62441 non-null  object 
 7   Fecha Inicio      62438 non-null  object 
 8   Fecha Termino     62441 non-null  object 
 9   Duración días     62441 non-null  object 
 10  Tipo de incendio  62441 non-null  object 
 11  Tipo Vegetación   62441 non-null  object 
 12  Régimen de fuego  62441 non-null  object 
 13  Tipo impacto      62441 non-null  object 
 14  Arbolado Adulto   54758 non-null  float64
 15  Renuevo           61506 non-null  float64
 16  Arbustivo         61918 non-null  float6

In [27]:
df["Duración días"].unique()

array(['2 a 3 Días', '1 Día', '4 a 7 Días', '0', 'Mayor de 7 Días', nan],
      dtype=object)

In [25]:
df[df["Duración días"].isnull()]

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
2064,2015,24.788611,-100.130833,Nuevo León,Galeana,Noreste,Ejido La Laguna,21/04/2015,21/04/2015,NaN,...,0.0,0.0,0.05,0.05,0.1,0 a 5 Hectáreas,01:00:00 a. m.,12:40:00 a. m.,02:45:00 a. m.,Naturales


'1 Día'